# BioData Catalyst Data Release QA
Validation tests in this notebook:
1. [**Patient counts of new studies**](https://basicnotebookinstance-rl0ytn08jb87.notebook.us-east-1.sagemaker.aws/notebooks/biodatacatalyst-pic-sure/access-dashboard-metadata/biodatacatalyst_data_release_QA.ipynb#Validation:-New-study-patient-counts): Patient counts of the new studies from the integration environment are compared to the patient counts in Patient_Count_Per_Consents.csv
2. [**Data dictionary comparison**](https://basicnotebookinstance-rl0ytn08jb87.notebook.us-east-1.sagemaker.aws/notebooks/biodatacatalyst-pic-sure/access-dashboard-metadata/biodatacatalyst_data_release_QA.ipynb#Validation:-Data-dictionary-comparison): Integration and production data dictionaries are compared to ensure complete match

### Prerequisites
- Developer access to the integration enviroment (token)
- Consent value(s) of the new study (or studies) to validate (phs number)
- Knowledge on whether a harmonized study was added

### Install packages

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds.git
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
import json
from pprint import pprint

import pandas as pd
import math

from shutil import copyfile
import PicSureClient
import PicSureBdcAdapter

from utils import get_full_consent_vals, compare_datadict_indices, get_topmed_and_harmonized_consents

### Connect to PIC-SURE
Be sure to use the **developer token** from the **integration environment**. It is necessary to have access to all studies to validate the counts.

In [ ]:
PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
resource_id = "70c837be-5ffc-11eb-ae93-0242ac130002" # Be sure to use Open Access resource id
token_file = "int_token.txt"

with open(token_file, "r") as f:
    my_token = f.read()
    
client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, my_token, True)

In [ ]:
bdc = PicSureBdcAdapter.Adapter(PICSURE_network_URL, my_token)
dictionary = bdc.useDictionary().dictionary()
resource = bdc.useResource(resource_id)

## Validation: New study patient counts
The purpose of this section is to validate the patient counts for newly ingested studies.

### Specify the new study (or studies) to be tested
To validate the new studies ingested, specify the phs numbers in the cell below without the consent group. 
For example, if the study of interest is the AMISH study, list `phs000956` below (*not* `phs000956.c1`).

In [ ]:
# to_validate = ['list', 'phs_numbers', 'here']
to_validate = ['phs002694', #ACTIV4a,
               'phs002710', #ACTIV4b
               'phs002752' #C3PO
              ]

to_validate_topmed, to_validate_harmonized = get_topmed_and_harmonized_consents(to_validate)

### Get patient count file from S3 bucket
This notebook uses `Patient_Count_Per_Consents.csv` from the S3 bucket as the reference file. First we need to copy this file over to this directory.

In [ ]:
src = '/home/ec2-user/SageMaker/studies/ALL-avillach-73-bdcatalyst-etl/general/completed/Patient_Count_Per_Consents.csv'
dst = '/home/ec2-user/SageMaker/biodatacatalyst-pic-sure/access-dashboard-metadata/Patient_Count_Per_Consents.csv'
#copyfile(src, dst)

In [ ]:
# Load S3 file as a dataframe in the Jupyter Notebook
patient_ref_file = pd.read_csv('Patient_Count_Per_Consents.csv', header=None, names=['consent', 'patient_count'])
patient_ref_file

In [ ]:
# Extract the consent groups based on the user-identified phs values
full_phs = get_full_consent_vals(to_validate, patient_ref_file)
full_phs

### Get patient count for the specified consent groups
Now the consent groups will be used to find the patient counts currently in the integration environment.

In [ ]:
# Start a new query and initialize output dictionary
patient_count_query = resource.query()
output = {}


for consentGroup in full_phs:
    print(consentGroup)
    patient_count_query.filter().delete("\\_consents\\") # Delete all consents
    patient_count_query.filter().add("\\_consents\\", consentGroup) # Add back consent group of interest
    patient_count_query.show()
    patient_count = patient_count_query.getCount() # Get patient count
    output[consentGroup] = patient_count # Add to output

In [ ]:
output

### Compare the values to the reference file
Finally we will compare the values from the reference file to the counts in the integration environment.

In [ ]:
for consent_val in output.keys():
    ref_count = int(patient_ref_file[patient_ref_file['consent'] == consent_val]['patient_count']) # Count from reference file
    integration_count = output[consent_val] # Count from integration environment
    # Display result message
    if ref_count == integration_count:
        print(consent_val, "passes validation")
    else:
        print('***DID NOT PASS VALIDATION:', consent_val)
        print('Expected count from Patient_Count_Per_Consents.csv:\t', ref_count)
        print('Count retrieved from integration environment:\t', integration_count)

## Validation: Data dictionary comparison
The purpose of this section is to compare the data dictionaries of the production and integration environments. These data dictionaries should be identical besides the studies that are being loaded and/or updated.

### Establish connection to production environment

In [ ]:
prod_PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure" 
prod_resource_id = "70c837be-5ffc-11eb-ae93-0242ac130002" # Be sure to use Open Access resource id
prod_token_file = "prod_token.txt"

with open(prod_token_file, "r") as f:
    prod_token = f.read()
    
prod_client = PicSureClient.Client()
prod_connection = prod_client.connect(prod_PICSURE_network_URL, prod_token, True)

In [ ]:
prod_bdc = PicSureBdcAdapter.Adapter(prod_PICSURE_network_URL, prod_token)
prod_resource = prod_bdc.useResource(prod_resource_id)

### Load data dictionaries
Next we will load the data dictionaries from the integration and production environments as dataframes. These will be used to compare the environments.

In [ ]:
int_dictionary = bdc.useDictionary().dictionary()
prod_dictionary = prod_bdc.useDictionary().dictionary()

In [ ]:
integ = int_dictionary.find().dataframe()

In [ ]:
prod = prod_dictionary.find().dataframe()

### Compare dictionaries
The following comparisons will be made between the dictionaries:
1. Find concept paths that exist in integration, but not production
2. Find concept paths that exist in production, but not integration
3. Identify differences in the dataframe between integration and production

The first comparisons use the concept paths, which we will extract and compare now.

In [ ]:
first_comparison = compare_datadict_indices(integ, prod, 1, to_validate)

In [ ]:
second_comparison = compare_datadict_indices(prod, integ, 2)

The third comparison compares the data in the data dictionary. The following function iterates through each row of the data dictionary and compares the integration and production results.

In [ ]:
harmonized = get_full_consent_vals(to_validate_harmonized, patient_ref_file)
topmed = get_full_consent_vals(to_validate_topmed, patient_ref_file)

In [ ]:
# Method to Compare Data Dictionaries

In [ ]:
# 1) remove rows from integ that contain variables from the user supplied phs (to_validate)

# only run if checking studies that are ONLY in integration

for phs in to_validate:
    print(phs)
    integ = integ[integ["studyId"].str.contains(phs)==False]

In [ ]:
# 2) if # of harmonized studies in prod and integ are different,
# remove rows containing DCC Harmonized (however that is listed) in BOTH prod and integ
# Get the expected differences in patient counts for each group

diff_total = patient_ref_file[patient_ref_file['consent'].isin(full_phs)].sum()['patient_count']

if len(harmonized)>0:
    #harmonized_diff_total = patient_ref_file[patient_ref_file['consent'].isin(harmonized)].sum()['patient_count']
    integ = integ[integ["studyId"].str.contains("DCC Harmonized data set")==False]
    prod = prod[prod["studyId"].str.contains("DCC Harmonized data set")==False]
    

In [ ]:
# 3) ensure column order is the same and concept path columns contain the same data
if False in (prod.columns == integ.columns):
    print("Columns in integration and production are different")

In [ ]:
# 4) order the dataframes by concept path column
#    check if the studyId columns are equal before moving forward
prod = prod.sort_values("studyId")
print("prod sorted")
integ = integ.sort_values("studyId")
print("integ sorted")

integ = integ.reset_index()
prod = prod.reset_index()


if(prod["studyId"].equals(integ["studyId"])):
    print("Study Id columns are identical, continue with QA")
else:
    print("Study Id columns are not the same, investigate why")

In [ ]:
# 5) Compare all fields of the Data Dictionaries
diff = integ.compare(prod)

if len(diff) == 0:
    print("integration and Production Data Dictionaries are identical")
else:
    diff

In [ ]:
# IGNORE for now
# For next QA process:
# Run counts for each value under _studies_consents - compare between Integration and Production

# Do patient counts for the following, if there is a change, be able to explain:
# All _studies have values for _studies_consents and _consents (_parent_consents, _topmed_consents)
# All top-level paths for studies have values in _consents and _studies_consents and _studies (_parent_consents, _topmed_consents)



# Create a table for each environment
# Compare the tables